In [ ]:
import os
import sys
sys.path.insert(0, "/data/zeljko/projects/medgpt/")

os.environ['HF_DATASETS_CACHE'] = "/data/zeljko/.cache/huggingface"
os.environ['TRANSFORMERS_CACHE'] = "/data/zeljko/.cache/huggingface"

%load_ext autoreload
%autoreload 2

In [ ]:
import pickle
import pandas as pd
from tokenizers.pre_tokenizers import WhitespaceSplit
from medgpt.config import Config

In [ ]:
config = Config(yaml_path='/home/ubuntu/projects/medgpt/configs/mimic.yaml')

In [ ]:
df = pd.read_csv('/data/zeljko/data/mimic/raw/noteevents.csv')

### doc2text

In [ ]:
doc2text = {str(doc):text for doc, text in df[['row_id', 'text']].values}

In [ ]:
pickle.dump(doc2text, open(config.path.dataset.doc2text, 'wb'))

### doc2info

In [ ]:
doc2info = {}

In [ ]:
df['chartdate'] = pd.to_datetime(df.chartdate)

In [ ]:
for id, subject_id, chartdate in df[['row_id', 'subject_id', 'chartdate']].values:
    doc2info[str(id)] = {'pt': str(subject_id), 'time': chartdate}

In [ ]:
pickle.dump(doc2info, open(config.path.dataset.doc2info, 'wb'))

### pt2info

In [ ]:
PT_CSV_PATH = "/home/ubuntu/data/mimic/PATIENTS.csv"
ADM_CSV_PATH = "/home/ubuntu/data/mimic/ADMISSIONS.csv"

In [ ]:
df = pd.read_csv(PT_CSV_PATH)

In [ ]:
df['DOB'] = pd.to_datetime(df.DOB)
df['DOD'] = pd.to_datetime(df.DOD)

In [ ]:
pt2info = {}
for id, row in df.iterrows():
    pt = str(row['SUBJECT_ID'])
    sex = 'Unknown'
    if row['GENDER'] == 'F':
        sex = 'Female'
    elif row['GENDER'] == 'M':
        sex = 'Male'
    
    dod = row['DOD']
    if pd.isna(row['DOD']):
        dod = None
    
    dob = row['DOB']
    if pd.isna(row['DOB']):
        dob = None
    pt2info[pt] = {'Sex': sex,
                   'DOB': dob,
                   'DOD': dod}

In [ ]:
df = pd.read_csv(ADM_CSV_PATH)

In [ ]:
for id, row in df.iterrows():
    eth = 'Unknown'
    if pd.notna(row.ETHNICITY):
        eth = row.ETHNICITY
    pt2info[str(row['SUBJECT_ID'])]['eth'] = eth

In [ ]:
pickle.dump(pt2info, open(config.path.dataset.pt2info, 'wb'))